In [21]:
import pandas as pd
from utility_functions import get_acc, add_row_to_csv

In [22]:
dataset = 'systematic'
model = 'gru'
n_best_models = 4

In [23]:
cv_df = pd.read_csv(f"model/RNN/report_{dataset}_{model}.csv")
evaluation_df = pd.read_csv('training_data/' + dataset + '/evaluation.csv')
fold_df = pd.read_csv('training_data/' + dataset + '/folds.csv')

In [24]:
for test_fold in sorted(fold_df['fold'].unique()):
    # Select the rows corresponding to the current fold
    df_fold = cv_df[cv_df['test_fold'] == test_fold]
    
    # Select evaluation dataframe for the fold based on sequenceID
    eval_df = evaluation_df[evaluation_df['sequenceID'].isin(fold_df[fold_df['fold'] == test_fold]['sequenceID'])]
    
    # Get the top n rows with minimum validation loss
    top_rows = df_fold.nsmallest(n_best_models, 'val_loss')

    # Initialize an empty list to store the predictions from the n models
    pred_list = []

    for _, row in top_rows.iterrows():
        n_layer = row['num_layers']
        h_size = row['hidden_size']
        # Read the predictions file for each model
        pred_df = pd.read_csv(f'model/RNN/predictions/{model}_{dataset}_{n_layer}layers_{h_size}features_fold{test_fold}.csv')
        pred_df.fillna(0, inplace=True)
        # Append the predictions for 'llda' to the list
        pred_list.append(pred_df['llda'])

    # Compute the average of the n models' predictions
    final_pred = sum(pred_list) / len(pred_list)
    
    # Replace 'llda' column with the averaged predictions in the first pred_df
    pred_df['llda'] = final_pred
    
    # Calculate accuracy using the averaged predictions
    acc = get_acc(eval_df, pred_df)
    
    # Save the accuracy to the CSV
    add_row_to_csv('acc_rate_csvs/' + dataset + '.csv', ['method', 'fold', 'acc'], [model, test_fold, acc])